# Количество деревьев
## 1. (10 баллов) Сколько существует возможных укорененных и неукорененных топологий деревьев на N листьях?

### Укорененные деревья
Укорененное бинарное дерево на $n$ листьях может быть сформировано путем добавления $n$-го листа 
в середине любого из ребер укорененного дерева на $n - 1$ листьях.
Для дерева на $n-1$ листьях сузетсвует $2n-3$ рёбер, к которым может быть присоеденен новый лист: $2n-4$ рёбер и корень.
Значит $R(n) = (2n-3)R(n-1)$.
Зная базу индукции получаем итоговую формулу:
$$R(n) = 1 \cdot 3 \cdot 5 \cdot \ldots \cdot (2n-3) = (2n-3)!!$$

### Неукорененные деревья
Аналогично, только количетво подходящиъ рёбер равно $2n-5$ на каждом шаге:
$$U(n)=1 \cdot 3 \cdot 5 \cdot \ldots \cdot (2n-5) = (2n-5)!!$$


# Построение деревьев

## (18 баллов) Напишите программу, реализующую алгоритмы WPGMA и UPGMA.

In [1]:
from pprint import pprint
import numpy as np

In [2]:
def wpgma(ds):
    while len(ds) > 1:
        old_node1, old_node2 = min(ds, key=ds.get)
        new_node = (old_node1, old_node2)
        
        for node in set([n for ns in ds.keys() for n in ns]) - {old_node1, old_node2}:
            ds[frozenset([new_node, node])] = (ds[frozenset([old_node1, node])] + ds[frozenset([old_node2, node])]) / 2
            
        ds = {k: v for k, v in ds.items() if (old_node1 not in k and old_node2 not in k)}
    return str(next(iter(ds))).replace('frozenset', '').replace('}', ''). replace('{', '')

def upgma(ds):
    cluster_size = lambda n: str(n).count('(') + 1 # :)
    
    while len(ds) > 1:
        old_node1, old_node2 = min(ds, key=ds.get)
        ons1, ons2 = cluster_size(old_node1), cluster_size(old_node2)
        
        new_node = (old_node1, old_node2)

        for node in set([n for ns in ds.keys() for n in ns]) - {old_node1, old_node2}:
            ds[frozenset([new_node, node])] = \
                (ds[frozenset([old_node1, node])] * ons1 + ds[frozenset([old_node2, node])] * ons2) / (ons1 + ons2)

        ds = {k: v for k, v in ds.items() if (old_node1 not in k and old_node2 not in k)}
    return str(next(iter(ds))).replace('frozenset', '').replace('}', ''). replace('{', '')

# Построение деревьев

## (18 баллов) Напишите программу, реализующую алгоритмы WPGMA и UPGMA.


In [3]:
def nj(ds):
    def m(ds, ns):
        n1, n2 = ns
        return np.mean([d for ns, d in ds.items() if n1 in ns and not n2 in ns])
    
    while len(ds) > 1:
        new_ds = {ns: (d - m(ds, ns) - m(ds, ns)) for ns, d in ds.items()}
        
        old_node1, old_node2 = min(ds, key=new_ds.get)
        new_node = (old_node1, old_node2)
        
        for node in set([n for ns in ds.keys() for n in ns]) - {old_node1, old_node2}:
            ds[frozenset([new_node, node])] = \
                (ds[frozenset([old_node1, node])] + ds[frozenset([old_node2, node])] - ds[frozenset([old_node1, old_node2])]) / 2
            
        ds = {k: v for k, v in ds.items() if (old_node1 not in k and old_node2 not in k)}
        
    return str(next(iter(ds))).replace('frozenset', '').replace('}', ''). replace('{', '')


## Тесты (для обеих задач):
## Тест 1

In [4]:
# frozenset for not caring about order of element (can't use simple set because it's mutable and unhashable)
t1 = {
    frozenset(['A', 'B']): 16,
    frozenset(['A', 'C']): 16,
    frozenset(['A', 'D']): 10,
    frozenset(['B', 'C']): 8,
    frozenset(['B', 'D']): 8,
    frozenset(['C', 'D']): 4
}

print('WPGMA:', wpgma(dict(t1)))
print('UPGMA:', upgma(dict(t1)))
print('Neighbor joining:', nj(dict(t1)))

WPGMA: (('B', ('C', 'D')), 'A')
UPGMA: (('B', ('C', 'D')), 'A')
Neighbor joining: (('B', ('C', 'D')), 'A')


## Тесты (для обеих задач):
## Тест 2

In [5]:
t2 = {
    frozenset(['A', 'B']): 5,
    frozenset(['A', 'C']): 4,
    frozenset(['A', 'D']): 7,
    frozenset(['A', 'E']): 6,
    frozenset(['A', 'F']): 8,
    frozenset(['B', 'C']): 7,
    frozenset(['B', 'D']): 10,
    frozenset(['B', 'E']): 9,
    frozenset(['B', 'F']): 11,
    frozenset(['C', 'D']): 7,
    frozenset(['C', 'E']): 6,
    frozenset(['C', 'F']): 8,
    frozenset(['D', 'E']): 5,
    frozenset(['D', 'F']): 9,
    frozenset(['E', 'F']): 8
}

print('WPGMA:', wpgma(dict(t2)))
print('UPGMA:', upgma(dict(t2)))
print('Neighbor joining:', nj(dict(t2)))

WPGMA: ('F', (('B', ('C', 'A')), ('E', 'D')))
UPGMA: ('F', (('B', ('C', 'A')), ('E', 'D')))
Neighbor joining: (('E', ('F', (('B', 'A'), 'C'))), 'D')
